## Computer Vision Algorithm for Tracking Planarian Motion
developed by Hokin Deng @xueqiandeng@yahoo.com

# Import Dependencies and Data

Import packages and make sure of the python technicality

In [2]:
import sys
import os
print("Python Version:", sys.version)
print("Python Executable:", sys.executable)
print("Python Path:", sys.path)

Python Version: 3.11.4 (main, Jul  5 2023, 08:41:25) [Clang 14.0.6 ]
Python Executable: /Users/billdeng/anaconda3/envs/unlearning_Version1/bin/python
Python Path: ['/Users/billdeng/PycharmProjects/unicellular', '/Users/billdeng/PycharmProjects/unicellular', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python311.zip', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11/lib-dynload', '', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11/site-packages']


In [3]:
import cv2
import math
import copy
import numpy as np
from collections import deque
import numpy as np
import argparse
import imutils

The video data should be named as "sample.avi" and put in the folder as the notebook.

In [4]:
# Open a video file (replace 'sample.avi' with the path)
cap = cv2.VideoCapture('sample.avi')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

Display the video if want to have a look at it

In [5]:
#cv2.startWindowThread()
# Loop to read and display frames
#while True:
    # Read a frame from the video
#    ret, frame = cap.read()
    # If the video has ended, break out of the loop
#    if not ret:
#        break
    # Display the frame in a window
#    cv2.imshow('Video', frame)
    # Exit the loop if the 'q' key is pressed
#    if cv2.waitKey(25) & 0xFF == ord('q'):
#        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

## Processing

First, get basic properties about our video

In [6]:
# Get basic video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
video_duration_sec = frame_count / frame_rate if frame_rate > 0 else 0

# Print the video properties
print(f"Frame Count: {frame_count}")
print(f"Frame Width: {frame_width}")
print(f"Frame Height: {frame_height}")
print(f"Frame Rate: {frame_rate} frames per second")
print(f"Video Duration: {video_duration_sec:.2f} seconds")

cap.release();

Frame Count: 0
Frame Width: 0
Frame Height: 0
Frame Rate: 0.0 frames per second
Video Duration: 0.00 seconds


Background subtraction

In [6]:
# Initialize video capture
cap = cv2.VideoCapture('sample.avi')

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('background_remove.mp4', fourcc, frame_rate, (frame_width, frame_height), False)

# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()

# Buffer for temporal smoothing
buffer_size = 5
frame_buffer = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Background subtraction
    fgMask = backSub.apply(frame)

    # Spatial smoothing (Gaussian blur)
    #blurred = cv2.GaussianBlur(fgMask, (5, 5), 0)

    # Add frame to buffer for temporal smoothing
    #frame_buffer.append(blurred)
    #if len(frame_buffer) > buffer_size:
    #    frame_buffer.pop(0)

    # Temporal smoothing (average of frames in buffer)
    #temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)

    # Write frame to video
    out.write(fgMask)

    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
out.release()
cv2.destroyAllWindows()

Spatial Smoothing

In [7]:
# Initialize video capture
cap = cv2.VideoCapture('background_remove.mp4')

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('background_remove_spatial_smooth.mp4', fourcc, frame_rate, (frame_width, frame_height), False)

# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()

# Buffer for temporal smoothing
buffer_size = 5
frame_buffer = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Background subtraction
    # fgMask = backSub.apply(frame)

    # Spatial smoothing (Gaussian blur)
    blurred = cv2.GaussianBlur(fgMask, (5, 5), 0)

    # Add frame to buffer for temporal smoothing
    # frame_buffer.append(blurred)
    # if len(frame_buffer) > buffer_size:
    #    frame_buffer.pop(0)

    # Temporal smoothing (average of frames in buffer)
   #  temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)

    # Write frame to video
    out.write(blurred)

    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
out.release()
cv2.destroyAllWindows()

Temporal Smoothing

In [11]:
# Initialize video capture
cap = cv2.VideoCapture('background_remove_spatial_smooth.mp4')

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('b_r_s_s_t_s.mp4', fourcc, frame_rate, (frame_width, frame_height), False)

# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()

# Buffer for temporal smoothing
buffer_size = 5
frame_buffer = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Background subtraction
    #fgMask = backSub.apply(frame)

    # Spatial smoothing (Gaussian blur)
    #blurred = cv2.GaussianBlur(fgMask, (5, 5), 0)

    # Add frame to buffer for temporal smoothing
    frame_buffer.append(blurred)
    if len(frame_buffer) > buffer_size:
        frame_buffer.pop(0)

    # Temporal smoothing (average of frames in buffer)
    temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)

    # Write frame to video
    out.write(temp_smoothed)

    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

dense opti flow

In [9]:
# Initialize video capture
cap = cv2.VideoCapture('')

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('dense_opti_flow.mp4', fourcc, frame_rate, (frame_width, frame_height), False)

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Error reading video")
    cap.release()
    exit()

# Convert to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while True:
    # Read next frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute magnitude and angle of the flow
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Create a binary mask where magnitude is greater than a threshold
    motion_mask = np.uint8(magnitude > 2)  # Adjust the threshold as needed



    # Display the motion mask
    # cv2.imshow('Motion Mask', motion_mask * 255)

    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Update previous frame and gray scale image
    prev_gray = gray

# Release video capture
cap.release()
cv2.destroyAllWindows()


OpenCV: Couldn't read video stream from file "background_remove_spatial_smooth_temporal_smooth.mp4"


Error reading video


error: OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


Define a function for video to image and use it

In [9]:
# Define a function to extract and save frames from a video file
current_location = os.getcwd();
output_folder = current_location + '/raw_images'
video_path = current_location + '/sample.avi'

def video2image(video_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the video
    video = cv2.VideoCapture(video_path)

    # Initialize frame count
    count = 0

    # Iterate through video frames
    while True:
        # Read a frame
        success, frame = video.read()
        # Break if no frame is read (end of video)
        if not success:
            break
        # Save the frame as an image
        cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), frame)
        # Increment frame count
        count += 1

    # Release the video object
    video.release()

    return count

Save the video into images

In [10]:
frame_count = video2image(video_path, output_folder)